<a href="https://colab.research.google.com/github/nataindata/ai-data-engineering-project/blob/main/Nataindata_BigQuery_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Data Engineering Project for Beginners: LangChain 🦜️🔗 + Vertex AI PaLM API on BigQuery


> "*AI is a big deal because it turns English into a programming language*"


This hands-on tutorial will show you how you can add generative AI features to your own applications with just a few lines of code using LangChain and LLMs on Google Cloud. We will build together a sample Python application that will be able to understand and respond to human language queries about the relational data stored in your Data warehouse.

* 🏆 This could be a great feature if you want to enable/showcase to your management how to talk to the data in a natural way and make their life easier.

* 🦾 It's basically creating your own AI Data assistant
---


## Objective

After completing the steps in this notebook:
- You will get a hands-on experience with using the open-source [LangChain framework](https://python.langchain.com/en/latest/index.html) to develop applications powered by large language models. LangChain makes it easier to develop and deploy applications against any LLM model in a vendor-agnostic manner.
- You will learn about the powerful features in [Google PaLM models made available through Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).


## The goal:

- Build a new AI-powered data chatbot, where users can describe their needs in simple English text, along with regular filters (like price, etc.)
- To add new generative AI experiences in their data warehouse for business stakeholders.


Dataset:
This notebook uses an example of TheLook data - a fictitious eCommerce clothing site developed by the Looker team.

- The dataset for this notebook has been sampled and created from a larger public retail dataset available at Bigquery `bigquery-public-data.thelook_ecommerce.inventory_items`.

## Before you begin

>⚠️ **Running this codelab will incur Google Cloud charges. You may also be billed for Vertex AI API usages.**

Pre-requisities:
- You need to have an active Google Cloud account to successfully complete this tutorial.
- Make a copy of the notebook and save a copy in the Drive. The account is the same as your Google Cloud account
- This sample notebook must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.
- You can use an existing project. Alternatively, you can create a new Cloud project [with free trial cloud credits.](https://cloud.google.com/free/docs/gcp-free-tier)
- At the end of the tutorial, you can optionally clean-up these resources to avoid further charges.


### Using this interactive notebook

Click the **run** icon on the top left corner ▶️  of each cell within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each cell to finish in their order before clicking the next “run” icon.

---

# Install Vertex AI LLM SDK


In [ ]:

! pip install --user --quiet google-cloud-aiplatform==1.25.0
! pip install --user --quiet google-api-python-client

# Install LangChain & SQL Alchemy
### LangChain is an open source framework that allows AI developers to combine LLM like PaLM with external sources of computation and data

[Langchain explanation here](https://pitch.com/v/unleashing-the-power-of-langchain-in-data-warehouse-jytu7h)

Large language models or LLMs such as ChatGPT/Vertex AI can answer questions about a lot of topics, but an LLM in isolation knows only what it was trained on, which doesn't include your personal/company data, such as if you're in a company and have proprietary documents not on the internet, as well as data or articles that were written after the LLM was trained.

So wouldn't it be useful if you or others such as your colleagues can
have a conversation with your own data and get questions answered using information from those documents and using an LLM?

LangChain is an open-source developer framework for building LLM applications. LangChain consists of several modular components as well as more end-to-end templates. The modular components in LangChain include:
- prompts,
- models,
- indexes,
- chains,
- and agents

In [ ]:
# Install Vertex AI LLM SDK, langchain and dependencies
! pip install --quiet google-cloud-aiplatform langchain==0.0.229 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0  transformers config --upgrade --user

### SQLAlchemy is an open-source SQL toolkit and object-relational mapper for the Python programming language released under the MIT License.

In [ ]:

# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet

---

#### ⚠️ Do not forget to "RESTART RUNTIME"

---

if you want to use this notebook go to file and then say "Save a copy" in Drive.
Here we're going to authenticate ourselves

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
PROJECT_ID = "sunny-emblem-404116"  # @param {type:"string"} ## Update this with your project id
LOCATION = "us-central1" # @param {type:"string"} ## Continue with us-central1

---

#### ⚠️ Do not forget to update the PROJECT_ID to your project

---

# Vertex AI LLM wrapper for using with Langchain

❗️In GCP don't forget to enable "Vertex AI API"

In [ ]:
import langchain
print(f"LangChain version: {langchain.__version__}")

# from a Vertex AI perspective you're getting the AI platform from Lang chain
from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

### Defining LangChain Models using Vertex AI PaLM API for Text


In [ ]:
import time
from typing import Any, Mapping, List, Dict, Optional, Tuple, Union
from pydantic import BaseModel, Extra, root_validator

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens


# Here we create utils classes for model text-bison@001
# We use private and public classes wrappers, to get the default parameters for calling Vertex AI API.

# Inheriting from pydantic BaseModel allows you to define the types of the
# class attributes (client, model_name, temperature, etc.).
# Pydantic will automatically validate the data against these types, helping catch potential errors early in the development process.

class _VertexCommon(BaseModel):
    """Wrapper around Vertex AI large language models.

    To use, you should have the
    ``google.cloud.aiplatform.private_preview.language_models`` python package
    installed.
    """
    client: Any = None #: :meta private:

    # Defining default model_name "text-bison@001"
    model_name: str = "text-bison@001"
    """Model name to use."""

    # temperature in the context of language models is a
    # parameter that influences the randomness and creativity
    temperature: float = 0.2
    """What sampling temperature to use."""

    top_p: int = 0.8
    """Total probability mass of tokens to consider at each step."""

    top_k: int = 40
    """The number of highest probability tokens to keep for top-k filtering."""

    max_output_tokens: int = 200
    """The maximum number of tokens to generate in the completion."""

    @property
    def _default_params(self) -> Mapping[str, Any]:
        """Get the default parameters for calling Vertex AI API."""
        return {
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "max_output_tokens": self.max_output_tokens
        }

    def _predict(self, prompt: str, stop: Optional[List[str]]) -> str:
        res = self.client.predict(prompt, **self._default_params)
        return self._enforce_stop_words(res.text, stop)

    def _enforce_stop_words(self, text: str, stop: Optional[List[str]]) -> str:
        if stop:
            return enforce_stop_tokens(text, stop)
        return text

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "vertex_ai"


# VertexLLM class for LLM text model "text-bison@001", inherits from private _VertexCommon class
# where model parameters are defined and from Langchain's LLM base class of Langchain
class VertexLLM(_VertexCommon, LLM):

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            from vertexai.preview.language_models import TextGenerationModel
        except ImportError:
            raise ValueError(
                "Could not import Vertex AI LLM python package. "
            )

        try:
            values["client"] = TextGenerationModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError(
                "Could not set Vertex Text Model client."
            )

        return values

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call out to Vertex AI's create endpoint.

        Args:
            prompt: The prompt to pass into the model.

        Returns:
            The string generated by the model.
        """
        return self._predict(prompt, stop)


# Create your input DataSet
- Copying data from public BigQuery `bigquery-public-data.thelook_ecommerce.inventory_items` dataset into a personal project.

TheLook is a fictitious eCommerce clothing site developed by the Looker team. The dataset contains information about customers, products, orders, logistics, web events and digital marketing campaigns. The contents of this dataset are synthetic

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import Client

client = Client(project=PROJECT_ID)

In [ ]:
# @title Specify Project details and location of the BQ table

project_id = "sunny-emblem-404116"  # @param {type:"string"}
location = "us-central1"  # @param {type:"string"}
dataset_id = "thelook_ecommerce" # @param {type:"string"}
table_name = "inventory_items" # @param {type:"string"}

In [ ]:
query = """
CREATE SCHEMA `{PROJECT_ID}.{dataset_id}`
OPTIONS(
  location="us"
  )
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
query = """
create or replace table `{PROJECT_ID}.{dataset_id}.{table_name}`
as
select
id,
product_id,
created_at,
sold_at,
cost,
product_category,
product_name,
product_brand,
product_retail_price,
product_department,
product_sku,
product_distribution_center_id
from `bigquery-public-data.thelook_ecommerce.inventory_items`
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id, table_name=table_name
)
query_job = client.query(query)
print(query_job.result())

## SQL Alchemy
SQL Alchemy provides you a method to create engines, write queries on top of any data set. In this case we are going to use BigQuery.

The engine is needed for SQL Alchemy to be able to query the project ID and the data set ID & needed as `engine` in Langchain's SQLDatabase. Aka proxy between BigQuery and Langchain

In [ ]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(table_uri)

In [ ]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name}`"""
engine.execute(query).first()


## SQLDatabaseChain
In LangChain, SQLDatabaseChain refers to a built-in chain that allows you to interact with SQL databases. It essentially enables you to bridge the gap between natural language and structured data stored in SQL databases.

Function:

- Allows you to query databases using natural language, similar to asking questions in plain English.
- Can be used to build chatbots, dashboards, and other applications that interact with SQL data.
- Supports various SQL dialects through SQLAlchemy, including MySQL, PostgreSQL, and Oracle.

To be able to query Tabular Data, we need to specify the connection, or `engine` and pass it through to an Langchain's SQLDatabase class
And then we'll create a chain that take our LLM, and DB. I'm setting `verbose=True` and `return_intermediate_steps=True` so you can see what is happening underneath the hood.

After, we will create a prompt template - it's one of the methods by which you could design a prompt and run it with parameters `top_k`, `table_info`, `input`

In [ ]:
from langchain import SQLDatabase, SQLDatabaseChain
from langchain.prompts.prompt import PromptTemplate

llm = VertexLLM(verbose=True)

def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[table_name])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

  #Define prompt for BigQuery SQL
  _googlesql_prompt = """You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  If someone asks for specific month, use ActivityDate between current month's start date and current month's end date

  Question: {input}"""

  # passing parameters into PromptTemplate, input variables & template
  BigQuerySQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k"],
      template=_googlesql_prompt,
  )

  # passing question to the prompt template
  final_prompt = BigQuerySQL_PROMPT.format(input=question, table_info=table_name, top_k=10000)

  # pass final prompt to SQL Chain
  # again the DB chain is the LLM initialized from and on top of the
  # database engine the SQL database instantiation and then DB chain to do a SQL Alchemy query
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][0]


In [ ]:
#Testing 1
bq_qna('Count total number of products which were sold')

In [ ]:
#Testing 2
bq_qna('what were the total sales for product with the name like Original Perry Suspenders in 2023')

In [ ]:
#Testing 3
bq_qna('Write an SQL query to extract the month from the date field, calculate the sum of monthly sales for product with the name like "Micro Slim Fit Boxer" in 2023, and display the results as product name, month name followed by sales for each month in 2023.')




**References:**

- Adapted from [LangChain Cookbook](https://github.com/gkamradt/langchain-tutorials) from [Greg Kamradt](https://twitter.com/GregKamradt)
- [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)
- [LangChain Python Documentation](https://python.langchain.com/en/latest/)
